In [1]:
# !pip install qrcode

In [2]:
import json
import uuid
import qrcode
from cryptography.fernet import Fernet
import random

random.seed(123)

# Generate a unique decentralized identifier (DID)
def generate_did():
    return f"did:example:{uuid.uuid4()}"

# Encrypt data to simulate cryptographic proof
def encrypt_data(data, key):
    f = Fernet(key)
    encrypted_data = f.encrypt(data.encode())
    return encrypted_data

# Decrypt data
def decrypt_data(encrypted_data, key):
    f = Fernet(key)
    decrypted_data = f.decrypt(encrypted_data).decode()
    return decrypted_data

# Create a verifiable credential
def create_verifiable_credential(student_info, issuer, key):
    credential = {
        "context": "https://www.w3.org/2018/credentials/v1",
        "type": ["VerifiableCredential"],
        "issuer": issuer,
        "credentialSubject": student_info,
    }
    serialized_credential = json.dumps(credential, indent=2)
    encrypted_credential = encrypt_data(serialized_credential, key)
    return encrypted_credential

# Generate QR Code for any text
def generate_qr_code(data, address):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    
    data = f'{address}/?data={data}'
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    img.save("did_qr_code.png")  # Saving the QR code to a file
    return "QR code generated and saved as did_qr_code.png"

# Main function to create a DID and VC
def main(student_info, address):
    # Generate cryptographic key
    key = Fernet.generate_key()
    
    # University and student info
    university_did = generate_did()
    
    # For demonstration
    # since we used all wallet information, we just have to keep 'id' key to simulate real implementation
    keys_to_keep = ['id']
    student_info = [{k: v for k, v in d.items() if k in keys_to_keep} for d in student_info]
    # Create the VC
    encrypted_vc = create_verifiable_credential(student_info, university_did, key)
    print("Encrypted Verifiable Credential:")
    print(encrypted_vc)
    
    # Decrypting the VC for display
    decrypted_vc = decrypt_data(encrypted_vc, key)
    print("\nDecrypted Verifiable Credential:")
    print(decrypted_vc) 

    # Generate QR Code for the DID
    qr_code_message = generate_qr_code(encrypted_vc, address)
    print(qr_code_message)

    return key

### generate wallets

In [3]:
random.seed(123)

s1= {"id": generate_did(),
     "name": "John Doe",
     "studentId": "UNINotts-123456",
     "course": "Computer Science",
    }

s2= {"id": generate_did(),
     "name": "Jane Doe",
     "studentId": "UNINotts-128223",
     "course": "Financial Technology",
    }

In [4]:
address = 'http://10.132.198.247:8501'

In [5]:
random.seed(123)

student_info = [s1,s2]
key = main(student_info, address)

Encrypted Verifiable Credential:
b'gAAAAABmM5MHKlBkhJGCkkkPtp_cKtL7WJfBUmqmRD4C78SNO-jS4tAENU-10L7nc3KDEUFjvVdheGGfWBZ_fRxsZCS8rz5Qdog_KyigJwPH74GuhO8Gh6sigIkP5_Jbt5govS-hfJfLb6IQVkEZX546hgoedoJ4biPMkgwNr1vQJjRzicH90Dkh7vHMccb51m29YmBdwfJw03Ah9OKGhQjUZ_R7q3-uZ_WDk9FRdp3AY_vNn2XgZBsvg1MSFhR0CQ9pXfGwnYhKBKmnlTsyP6_rD4WHnl-0o_IMNb6uuzoN-nqOhIrPORhSKhz7VwalBxc084adQzcOg3PpbiyJhLt9HIwf_FMQnros2Y3pPYPsupv-0XE2l9py4b8lnw2CrS4SL-KJIAf0voeRv2n1A_Ic3noaBmkVeeFoxrUPUOLMYMmh8UWMSAUfrhl_YsHU65e6XQ70IULGdJOq18jsVpvq88kFSUdMIg-kjz-lBY2AI-DIbSGgqIyyvXlbhmGCd8vs9ajvOh3eIEvRYDPWxEktB4ZeILlhXg=='

Decrypted Verifiable Credential:
{
  "context": "https://www.w3.org/2018/credentials/v1",
  "type": [
    "VerifiableCredential"
  ],
  "issuer": "did:example:c6726827-21e5-474f-acce-d97ad8274e1f",
  "credentialSubject": [
    {
      "id": "did:example:3ebeff4e-aa90-4a13-8892-e095eefb2a09"
    },
    {
      "id": "did:example:9499ce3a-fd2c-4e9c-bbd9-fecf524a0dad"
    }
  ]
}
QR code generated and saved as did

In [6]:
key

b'lEIrfomp6SCg-ejEvCZeyX4YpEvbMehJBjXYjjWDzWw='

### Reading qr code

In [7]:
# !pip install pillow

In [8]:
# !pip install opencv-python

In [9]:
# !pip install pyzbar

In [10]:
import cv2
from pyzbar import pyzbar

In [11]:
trial_img= cv2.imread('did_qr_code.png')
qcd = cv2.QRCodeDetector()
data, bbox, _ = qcd.detectAndDecode(trial_img)

In [12]:
data

"http://10.132.198.247:8501/?data=b'gAAAAABmM5MHKlBkhJGCkkkPtp_cKtL7WJfBUmqmRD4C78SNO-jS4tAENU-10L7nc3KDEUFjvVdheGGfWBZ_fRxsZCS8rz5Qdog_KyigJwPH74GuhO8Gh6sigIkP5_Jbt5govS-hfJfLb6IQVkEZX546hgoedoJ4biPMkgwNr1vQJjRzicH90Dkh7vHMccb51m29YmBdwfJw03Ah9OKGhQjUZ_R7q3-uZ_WDk9FRdp3AY_vNn2XgZBsvg1MSFhR0CQ9pXfGwnYhKBKmnlTsyP6_rD4WHnl-0o_IMNb6uuzoN-nqOhIrPORhSKhz7VwalBxc084adQzcOg3PpbiyJhLt9HIwf_FMQnros2Y3pPYPsupv-0XE2l9py4b8lnw2CrS4SL-KJIAf0voeRv2n1A_Ic3noaBmkVeeFoxrUPUOLMYMmh8UWMSAUfrhl_YsHU65e6XQ70IULGdJOq18jsVpvq88kFSUdMIg-kjz-lBY2AI-DIbSGgqIyyvXlbhmGCd8vs9ajvOh3eIEvRYDPWxEktB4ZeILlhXg=='"

### Verification process

In [13]:
def verify_credential(encrypted_credential, key):
    try:
        f = Fernet(key)
        decrypted_data = f.decrypt(encrypted_credential)
        credential = json.loads(decrypted_data.decode('utf-8'))  # Decode the bytes object to string
        
        if verify_signature(credential):
            print("Credential is valid and authentic.")
            return credential
        else:
            print("Invalid signature. Credential could be tampered with.")
            return None  # Explicitly return None when verification fails
    except Exception as e:
        print(f"Error during verification: {str(e)}")
        return None  # Explicitly return None when an exception occurs

def verify_signature(credential):
    # Dummy function for signature verification
    # In real implementation, this function should check the digital signature properly
    return True  # Assuming signature is always valid for demonstration

In [14]:
from urllib.parse import urlparse, parse_qs

data = parse_qs(urlparse(data).query)['data'][0][2:-1]
data = bytes(data, 'utf-8')

In [15]:
# Example usage
decrypted_credential = verify_credential(data, key)
decrypted_credential

Credential is valid and authentic.


{'context': 'https://www.w3.org/2018/credentials/v1',
 'type': ['VerifiableCredential'],
 'issuer': 'did:example:c6726827-21e5-474f-acce-d97ad8274e1f',
 'credentialSubject': [{'id': 'did:example:3ebeff4e-aa90-4a13-8892-e095eefb2a09'},
  {'id': 'did:example:9499ce3a-fd2c-4e9c-bbd9-fecf524a0dad'}]}

In [16]:
def authenticate(decrypted_credential, wallet):
    id_exists = any(s1['id'] == subj['id'] for subj in decrypted_credential['credentialSubject'])
    if id_exists:
        print('Authentication Success\n------------------------------------------\n')
        print(json.dumps(wallet, indent=2))
    else:
        print('Authentication Error')

In [17]:
authenticate(decrypted_credential,s1)

Authentication Success
------------------------------------------

{
  "id": "did:example:3ebeff4e-aa90-4a13-8892-e095eefb2a09",
  "name": "John Doe",
  "studentId": "UNINotts-123456",
  "course": "Computer Science"
}


### Saving wallets and key

In [18]:
import os
import json

# Create a folder if it doesn't exist
folder_path = "streamlit/wallets"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Write each dictionary to a separate file
for i, my_dict in enumerate([s1, s2], start=1):
    file_path = os.path.join(folder_path, f"wallet{i}.json")
    with open(file_path, 'w') as file:
        json.dump(my_dict, file)

print("Folder and files created successfully.")

Folder and files created successfully.


In [19]:
# Create a folder if it doesn't exist
folder_path = "streamlit/key"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Define the file path
file_path = os.path.join(folder_path, "key.bin")

# Write the byte string to the file
with open(file_path, 'wb') as file:
    file.write(key)

print("Byte string saved as file successfully.")

Byte string saved as file successfully.


### Creating executable streamlit app file

In [20]:
%%writefile streamlit/app.py

import streamlit as st
from cryptography.fernet import Fernet
import json
import uuid

# Function to extract query parameters
def get_query_params():
    query_params = st.query_params
    return query_params

# Function to read JSON file and parse as dictionary
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Authentication process
def verify_credential(encrypted_credential, key):
    try:
        f = Fernet(key)
        decrypted_data = f.decrypt(encrypted_credential)
        credential = json.loads(decrypted_data.decode('utf-8'))  # Decode the bytes object to string
        
        if verify_signature(credential):
            print("Credential is valid and authentic.")
            return credential
        else:
            print("Invalid signature. Credential could be tampered with.")
            return None  # Explicitly return None when verification fails
    except Exception as e:
        print(f"Error during verification: {str(e)}")
        return None  # Explicitly return None when an exception occurs

def verify_signature(credential):
    # Dummy function for signature verification
    # In real implementation, this function should check the digital signature properly
    return True  # Assuming signature is always valid for demonstration

def authenticate(decrypted_credential, wallet):
    id_exists = any(wallet['id'] == subj['id'] for subj in decrypted_credential['credentialSubject'])
    if id_exists:
        # print('Authentication Success\n------------------------------------------\n')
        # return json.dumps(wallet, indent=2)
        return wallet
    else:
        # print('Authentication Error')
        return 'No data'

# Main application
def main():
    st.title('NOTTS ID')
    
    # Get the data from the URL query parameter
    query_params = get_query_params()
    data = query_params.get('data', 'No data provided')+"'"
    data = bytes(data[2:-1], 'utf-8')
    
    # Display the decoded data
    st.header('Select information to share')
    
    # # Allow user to input a key
    # key = st.text_input("Enter Key:", "")
    # key = bytes(key, 'utf-8')
    
    # Sharing Data
    wallet = st.number_input('Enter wallet number:',1)
    useName = st.checkbox("Name")
    useStudentId = st.checkbox("Student ID")
    useCourse = st.checkbox("Course")
    
    # Read JSON file and parse as dictionary
    file_path_wallet = f"wallets/wallet{wallet}.json"
    s1 = read_json_file(file_path_wallet)
    
    # Read the byte string from the file
    file_path_key = f"key/key.bin"
    with open(file_path_key, 'rb') as file:
        key = file.read()
    
    # Process the data with the key
    if st.button("Process Data"):
        # st.write(data)
        # st.write(key)
        
        decrypted_credential = verify_credential(data, key)
        result = authenticate(decrypted_credential, s1)
        
        # Filtered dictionary
        all_keys = result.keys()
        keys_to_use = [False, useName, useStudentId, useCourse]
        filtered_dict = {key: value for key, value in result.items() if keys_to_use[list(all_keys).index(key)]}
        
        # Convert filtered dictionary to JSON
        json_data = json.dumps(filtered_dict)
        
        st.header('Result')
        st.json(json_data)
        


if __name__ == "__main__":
    main()

Writing streamlit/app.py


In [21]:
# !pip install Flask

In [22]:
# from flask import Flask, request, render_template

# app = Flask(__name__)

# @app.route('/decode')
# def decode():
#     data = request.args.get('data', '')
#     # You can add your decoding algorithm here
#     decoded_data = data  # Placeholder for your decoding process
#     return render_template('decoded.html', decoded_data=decoded_data)

# if __name__ == '__main__':
#     app.run(debug=True, host='0.0.0.0', use_reloader=False)


In [23]:
# !pip install streamlit

In [24]:
# import streamlit as st
# from urllib.parse import urlparse, parse_qs

# # Function to extract query parameters
# def get_query_params():
#     query_params = parse_qs(urlparse(st.experimental_get_query_params()).query)
#     return {key: value[0] for key, value in query_params.items()}

# # Main application
# def main():
#     st.title('QR Code Decoder')
    
#     # Get the data from the URL query parameter
#     query_params = get_query_params()
#     data = query_params.get('data', 'No data provided')
    
#     # Display the decoded data
#     st.header('Decoded Data')
#     st.write(data)
    
#     # Here, you could add your algorithm to further decode/process 'data'
#     # processed_data = your_decode_function(data)
#     # st.write(processed_data)

# if __name__ == "__main__":
#     main()